# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 9.37MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Usi

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [3]:
import pandas as pd
dataset = pd.read_csv("/content/Dataset.csv")
dataset.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [5]:
dataset.shape

(104, 2)

In [23]:
dataset.isnull().sum()

المخالفة    0
الغرامة     0
dtype: int64

In [24]:
dataset.duplicated().sum()

0

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [6]:
import os
import markdown

In [7]:
directory = 'markdown_files'
os.makedirs(directory, exist_ok=True)

In [8]:
len(dataset)

104

In [9]:
for i in range(len(dataset)):

    violation = dataset['المخالفة'].iloc[i]
    ticket = dataset['الغرامة'].iloc[i]

    markdown_content = f"**{violation}**- "
    markdown_content += f"{ticket}"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [10]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      markdown_texts.append(markdown_content)

In [11]:
markdown_texts

['**طمس لوحات المركبة.**- الغرامة المالية 3000 - 6000 ريال',
 '**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.**- الغرامة المالية 100 - 150 ريال',
 '**عدم ربط حزام الأمان.**- الغرامة المالية 150 - 300 ريال',
 '**استعمال المركبة لغير الغرض الذي رخصت من أجله.**- الغرامة المالية 500 - 900 ريال',
 '**القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.**- الغرامة المالية 1000 - 2000 ريال',
 '**عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز.**- الغرامة المالية 500 - 900 ريال',
 '**نقل الركاب في الأماكن غير المخصصة لهم في المركبة.**- الغرامة المالية 500 - 900 ريال',
 '**قيام السائق بتخطي أرتال المركبات أمام إشارات المرور أو نقاط الضبط الأمني باستخدام كتف الطريق أو المسار المخصص للالتفاف.**- الغرامة المالية 150 - 300 ريال',
 '**عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على المسارات المخصصة لها.**- الغرامة المالية 500 - 900 ريا

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap=50
)

texts= text_splitter.create_documents(markdown_texts)

In [13]:
texts[0]

Document(metadata={}, page_content='**طمس لوحات المركبة.**- الغرامة المالية 3000 - 6000 ريال')

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings = HuggingFaceEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")
db = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [15]:
from langchain.prompts import PromptTemplate
prompt_template ="""
INSTRUCTIONS:
You will give the user advice about traffic violation write the answer and advise with arabic language
Answer the question based only on the following context:
Context: {context}
Question: {question}
Your answer with advice:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [16]:
from langchain_groq import ChatGroq
api_key = "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"
llm = ChatGroq(
    temperature=0,
    groq_api_key=api_key,
    model_name="llama-3.1-70b-versatile"
    )

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [17]:
from langchain.chains import LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
    )

<ipython-input-17-f76fa031cad6>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [18]:
db.similarity_search_with_score

<bound method Chroma.similarity_search_with_score of <langchain_chroma.vectorstores.Chroma object at 0x7ed292faf9d0>>

In [19]:
db.similarity_search_with_score('ماهي الغرامة على القيادة بدون رخص')

[(Document(metadata={}, page_content='**قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.**- الغرامة المالية 1000 - 2000 ريال'),
  255.9075469970703),
 (Document(metadata={}, page_content='**عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.**- الغرامة المالية 150 - 300 ريال'),
  312.8961181640625),
 (Document(metadata={}, page_content='**قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.**- الغرامة المالية 1000 - 2000 ريال'),
  317.1308898925781),
 (Document(metadata={}, page_content='**قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.**- الغرامة المالية 500 - 900 ريال'),
  348.640380859375)]

In [20]:
def query_rag(query: str):

    similarity_search_results = db.similarity_search_with_score(query, k=2)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = chain.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [21]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

Prompt after formatting:

INSTRUCTIONS:
You will give the user advice about traffic violation 
Answer the question based only on the following context:
Context: **قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.**- الغرامة المالية 1000 - 2000 ريال

**عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.**- الغرامة المالية 150 - 300 ريال
Question: ماهي الغرامة على القيادة بدون رخصة؟
Your answer with advice:


> Finished chain.


{'context': '**قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.**- الغرامة المالية 1000 - 2000 ريال\n\n**عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.**- الغرامة المالية 150 - 300 ريال',
 'question': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'لا يوجد في السياق المحدد غرامة على القيادة بدون رخصة، ولكن يوجد غرامة على عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة، وتبلغ قيمتها بين 150 - 300 ريال.\n\nنصيحتي لك: تأكد دائمًا من حمل رخصة القيادة ورخصة السير أثناء القيادة لتجنب دفع غرامات مالية.'}

In [22]:
response = query_rag("ماهي غرامة التدخين أثناء القيادة؟")
response

Prompt after formatting:

INSTRUCTIONS:
You will give the user advice about traffic violation 
Answer the question based only on the following context:
Context: **قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.**- الغرامة المالية 1000 - 2000 ريال

**قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.**- الغرامة المالية 1000 - 2000 ريال
Question: ماهي غرامة التدخين أثناء القيادة؟
Your answer with advice:


> Finished chain.


{'context': '**قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.**- الغرامة المالية 1000 - 2000 ريال\n\n**قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.**- الغرامة المالية 1000 - 2000 ريال',
 'question': 'ماهي غرامة التدخين أثناء القيادة؟',
 'text': 'لا يوجد في السياق المحدد غرامة مالية محددة للتدخين أثناء القيادة. ومع ذلك، أود أن أوصي بعدم التدخين أثناء القيادة، حيث يمكن أن يؤدي ذلك إلى تشتيت الانتباه وزيادة خطر الحوادث. يُفضل دائمًا اتباع قواعد المرور والسلامة لضمان سلامتك وسلامة الآخرين على الطريق.'}